In [1]:
pip install gensim=='3.6.0'

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Note: you may need to restart the kernel to use updated packages.


In [2]:
import spacy
import gensim
from gensim.models import Word2Vec 
from nltk.tokenize import RegexpTokenizer
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
import numpy as np
import pandas as pd

In [3]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm
!python -m spacy download ru_core_news_sm

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/d

# Что вообще может spacy?

In [4]:
import spacy

In [5]:
nlp = spacy.load('ru_core_news_sm')

In [6]:

doc = nlp(u'Я принимала антидепрессанты 3 месяца, толку - ноль.\
Бросила пить, хоть в инструкции и стояло, что нельзя бросать резко,\
но я это сделала, так как вообще не видела никакого действия. Последствий никаких, как будто принимала не АД,\
а витамин С... Так что не верю в АД, хотя каждому своё, да и АД разные бвают.')
spacy.displacy.render(doc, style='dep', jupyter=True)

In [7]:
for token in doc:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])

Я nsubj принимала VERB []
принимала ROOT принимала VERB [Я, антидепрессанты, месяца, .]
антидепрессанты obj принимала VERB []
3 nummod:gov месяца NOUN []
месяца obl принимала VERB [3, толку, -, ноль]
, punct толку NOUN []
толку conj месяца NOUN [,]
- conj месяца NOUN []
ноль conj месяца NOUN []
. punct принимала VERB []
Бросила ROOT Бросила VERB [пить, стояло, .]
пить xcomp Бросила VERB [инструкции]
, punct инструкции NOUN []
хоть advmod инструкции NOUN []
в case инструкции NOUN []
инструкции obl пить VERB [,, хоть, в]
и cc стояло VERB []
стояло conj Бросила VERB [и, нельзя]
, punct нельзя ADV []
что mark нельзя ADV []
нельзя ccomp стояло VERB [,, что, бросать, сделала]
бросать csubj нельзя ADV [резко]
резко advmod бросать VERB []
, punct сделала VERB []
но cc сделала VERB []
я nsubj сделала VERB []
это obj сделала VERB []
сделала conj нельзя ADV [,, но, я, это, видела]
, punct видела VERB []
так mark видела VERB [как]
как fixed так ADV []
вообще advmod видела VERB []
не advmod видела 

# Выглядит вроде прилично, наверное можно это использовать для извлечения признаков

In [8]:
import string

In [9]:
for token in doc:
  if token.text not in string.punctuation:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

Я я PRON PRON nsubj X True True
принимала принимать VERB VERB ROOT xxxx True False
антидепрессанты антидепрессант NOUN NOUN obj xxxx True False
3 3 NUM NUM nummod:gov d False False
месяца месяц NOUN NOUN obl xxxx True False
толку толку NOUN NOUN conj xxxx True False
ноль ноль NOUN NOUN conj xxxx True False
Бросила бросить VERB VERB ROOT Xxxxx True False
пить пить VERB VERB xcomp xxxx True False
хоть хоть ADV ADV advmod xxxx True False
в в ADP ADP case x True True
инструкции инструкция NOUN NOUN obl xxxx True False
и и CCONJ CCONJ cc x True True
стояло стоять VERB VERB conj xxxx True False
что что SCONJ SCONJ mark xxx True True
нельзя нельзя ADV ADV ccomp xxxx True False
бросать бросать VERB VERB csubj xxxx True False
резко резко ADV ADV advmod xxxx True False
но но CCONJ CCONJ cc xx True True
я я PRON PRON nsubj x True True
это это PRON PRON obj xxx True True
сделала сделать VERB VERB conj xxxx True False
так так ADV ADV mark xxx True True
как как SCONJ SCONJ fixed xxx True True
вообще

# Но для извлечения морфологических признаков, наверное, лучше использовать pymotphy2

In [10]:
pip install pymorphy2[fast]

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Note: you may need to restart the kernel to use updated packages.


In [11]:
import pymorphy2

In [12]:
morph = pymorphy2.MorphAnalyzer()

In [13]:
doc = nlp(u'Кто принимал его в своей практике? Омерзительный препарат.')

In [14]:
doc[0]

Кто

In [15]:
morph.parse(doc[0].text)[0].tag._str

'NPRO,masc sing,nomn'

In [16]:
# dir(morph.parse(doc[0].text)[0].tag)

# Векторизация с помощью w2v

In [17]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/neuroleptic_ru.txt

--2021-04-12 23:15:50--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/neuroleptic_ru.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3638613 (3.5M) [text/plain]
Saving to: ‘neuroleptic_ru.txt.1’

neuroleptic_ru.txt. 100%[===================>]   3.47M  --.-KB/s    in 0.06s   

2021-04-12 23:15:50 (53.7 MB/s) - ‘neuroleptic_ru.txt.1’ saved [3638613/3638613]



# Одна из самых маленьких моделей cbow с rusvectores.org

In [18]:
!wget https://rusvectores.org/static/models/news_upos_cbow_300_2_2017.bin.gz

--2021-04-12 23:15:51--  https://rusvectores.org/static/models/news_upos_cbow_300_2_2017.bin.gz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 140310878 (134M) [application/x-gzip]
Saving to: ‘news_upos_cbow_300_2_2017.bin.gz.1’

news_upos_cbow_300_ 100%[===================>] 133.81M  56.8MB/s    in 2.4s    

2021-04-12 23:15:53 (56.8 MB/s) - ‘news_upos_cbow_300_2_2017.bin.gz.1’ saved [140310878/140310878]



In [19]:
handler = open('neuroleptic_ru.txt', 'r', encoding='utf-8')
my_txt = handler.read()
handler.close()

In [20]:
posts = str_list = my_txt.split('----')

In [21]:
tokenizer = RegexpTokenizer(r'\w+')
sentences_tokenized = [w.lower() for w in posts]
sentences_tokenized = [tokenizer.tokenize(i) for i in sentences_tokenized]

In [22]:
w2v = gensim.models.KeyedVectors.load_word2vec_format('news_upos_cbow_300_2_2017.bin.gz', binary=True)

In [27]:
len(w2v.vocab)

124590

In [23]:
model = Word2Vec(size=300, min_count=1)
model.build_vocab(sentences_tokenized)
total_examples = model.corpus_count
model.build_vocab([list(w2v.vocab.keys())], update=True)
model.intersect_word2vec_format("news_upos_cbow_300_2_2017.bin.gz", binary=True)
model.train(sentences_tokenized, total_examples=total_examples, epochs=model.iter)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  


(1321292, 1530870)

In [31]:
len(model.wv.vocab)

151409

In [37]:
w2v.save('wv')

In [39]:
model.wv.save('wv2')

In [40]:
!ls

Untitled-Copy1.ipynb		    tag.dict
Untitled.ipynb			    test.txt
Untitled1.ipynb			    train.txt
logs				    valid.txt
neuroleptic_ru.txt		    w2v
neuroleptic_ru.txt.1		    w2v.vectors.npy
news_upos_cbow_300_2_2017.bin.gz    wv
news_upos_cbow_300_2_2017.bin.gz.1  wv.vectors.npy
nltk_data			    wv2
onstart.log			    wv2.vectors.npy
onstart.sh


In [41]:
model.wv.most_similar('алимемазин')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('p', 0.9969420433044434),
 ('стимуляторов', 0.9965527057647705),
 ('большинство', 0.9964768290519714),
 ('обратного', 0.9964586496353149),
 ('витаминов', 0.9964235424995422),
 ('появляется', 0.9964066743850708),
 ('прошу', 0.996398389339447),
 ('следовательно', 0.9963942170143127),
 ('нервной', 0.9963140487670898),
 ('функции', 0.9962787628173828)]

In [42]:
cosine_similarity(np.atleast_2d(model.wv.get_vector('алимемазин')), \
                  np.atleast_2d(model.wv.get_vector('азалептин')))[0][0]

0.9855263

In [52]:
cosine_similarity(np.atleast_2d(model.wv.get_vector('алимемазин')), \
                  np.atleast_2d(model.wv.get_vector('стул')))[0][0]

0.92223203

In [53]:
posts[9]

'\nПринимал порядка 2-3 недель от 10 до 20 мг Алимемазина, совместно с 200 мг Азалептина и 30 мг Ремерона.Всё это принималось за полчаса до сна.Лично я не почувствовал действие Тералиджена, словно и не принимал его вовсе.Мою довольно средневзвешенную ипохондрию он не купировал,равно как и депрессивные симптомы.\n'

# Это для того, чтобы сделать файл, который потом удобно размечать

In [54]:
sentences_for_annts = [' '.join(sent) for sent in sentences_tokenized]
sentences_for_annts = [sent.split() for sent in set(sentences_for_annts)]
sents_for_annts= []
for sent in sentences_for_annts:
  sent.append('\n')
  sents_for_annts.extend(sent) 

In [55]:
len(sents_for_annts)

233133

In [56]:
df = pd.DataFrame(columns=['TOKEN', 'NER', 'COREFERENCE_CLUSTER'])

In [57]:
df['TOKEN'] = sents_for_annts
df['NER'] = [0 for i in range(len(sents_for_annts))]